# Model Deployment


This notebook will focus on how we can deploy a model to production, in fact we will do a deployment of the model trained in [this notebook](../internet_of_wands.ipynb).


## Deployment Types

There are different ways to deploy a model to production, mainly:
- Server API Endpoint (real-time): most common/generic solution, the prediction is carried out in a server -> very flexible, model can be only in server can also have reduced latencies in some cases
- Batch inference: at regular times or triggered manually the data is evaluated for a set of data -> useful for analysis but not suitable for real-time applications
- Edge deployment: the inference is carried out directly at the device (phone, computer, car, browser, etc) -> heavily depends on the device

## Dependencies

We will be using [FastAPI](https://fastapi.tiangolo.com/) to put together a simple prediction endpoint.

If not installed already in the environment in can be installed with pip as

```bash
pip install fastapi
pip install "uvicorn[standard]"
```

In [ ]:
%load_ext autoreload
%autoreload 2
import requests
import json
from pathlib import Path
import pandas as pd
import numpy as np


In [ ]:

# let us get some example data
with open(Path("../iow_data/alohomora/Abel_2dcdf140.da7cfe.json")) as f:
    example_data = json.load(f)

example_data

## Run server

We are going to be running a FastAPI server of the API. We can also server the static files of the application also with the same server. Please check the file [`server.py`](server.py).

The server can be run with the following command in this folder:

```bash
uvicorn server:app --reload
```


## Collect endpoint

We are going to focuss on predicting a model but to warm up and getting familiar with APIs we can checkout and play with a collect endpoint that more or less does the same than the MQTT process but in a simpler way.

Checkout the predict endpoint in [`server.py`](server.py).

In [ ]:
# we can test our endpoint collect by sending the data to it
requests.put(json=example_data, url="http://localhost:8000/api/collect")

## Predict endpoint

The predict endpoint is bit more endpoint, as we have have to reproduce all the transformations steps and evaluations steps in the training.

In [ ]:
from schemas import PredictSample
from model import read_sample, extract_features, PredictModel

In [ ]:
# we can create a predict example to test our transformations
predict_sample = PredictSample(**example_data)
predict_sample.dict()

In [ ]:
# we can test first the read_sample function
eval_df = read_sample(predict_sample)
eval_df

In [ ]:
# then we can test the extract_features function
eval_df_extra = extract_features(eval_df)
eval_df_extra

In [ ]:
# finally we can test the whole predict function
model = PredictModel("../optimized_gb_clf.joblib")

model.predict(predict_sample)

In [ ]:
# also the endpoint instead of returning a string returns a json
response = requests.post(json=predict_sample.dict(), url="http://localhost:8000/api/predict")
response.json()